### Problem Solving Agents & SEARCH (“Six Degrees of Kevin Bacon”)

In [ ]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [ ]:
import csv
import sys

In [ ]:
names = {}
people = {}
movies = {}

**Importing CSV data into adjacency matrix list**

In [ ]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

**Breadth First Search Main Function**

In [ ]:
def main():
    
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path_bfs(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


**Shortest Path BFS**

In [ ]:
def shortest_path_bfs(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """

    solution = list()
    explored = set()

    solution_found = False
    empty = False

    start = Node(state=source, parent=None, action=None)
    frontier = QueueFrontier()
    frontier.add(start)

    while not solution_found:
        if frontier.empty():
            solution_found = True
            empty = True
        
        # Choose a node from frontier
        node = frontier.remove()

        # If node is the target, then we have a solution
        if node.state == target:
            solution_found = True
            while node.parent is not None:
                pid, mid = node.state, node.action
                solution.append((mid, pid))
                node = node.parent
            solution.reverse()

        # Mark node as explored
        explored.add(node)
        neighbors = neighbors_for_person(node.state)

        for neighbor in neighbors:
            child = Node(state=neighbor[1], action=neighbor[0], parent=node)
            # Add neighbor to frontier
            frontier.add(child)

            # If any child node from neighbors is the target, then we have a solution
            if child.state == target:
                solution_found = True
                while child.parent is not None:
                    pid, mid = child.state, child.action
                    solution.append((mid, pid))
                    child = child.parent
                solution.reverse()

    if solution_found:
        if empty:
            return None
        return solution

In [ ]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [ ]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


In [ ]:
if __name__ == "__main__":
    main()

Loading data...
Data loaded.
Name: Tom Hanks
Name: Tom Cruise
2 degrees of separation.
1: Tom Hanks and Max von Sydow starred in Extremely Loud & Incredibly Close
2: Max von Sydow and Tom Cruise starred in Minority Report


**Depth First Search Shortest Path Function**

In [ ]:
def shortest_path_dfs(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """
    count_explored = 0
    print(f"Searching: {count_explored}", end="", flush=True)

    # Initiate starting node and frontier
    start = Node(state=source, parent=None, action=None)
    frontier = StackFrontier()
    frontier.add(start)

    explored = set()

    while True:
        if frontier.empty(): # If frontier is empty, then no solution
            print()
            return None
        
        # Checkout node from frontier
        node = frontier.remove()

        # Update count
        count_explored += 1
        print(f"\rSearching: {count_explored}", end="", flush=True)            

        # Mark node as explored
        explored.add(node.state)

        # Add neighbors to frontier
        for action, state in neighbors_for_person(node.state):
            if not frontier.contains_state(state) and state not in explored:
                child = Node(state=state, parent=node, action=action)

                # If child is the target, get solution
                if child.state == target:
                    node = child
                    solution = []

                    # Loop backwards to find solution
                    while node.parent is not None:
                        solution.append((node.action, node.state))
                        node = node.parent
                    # print newline
                    print()

                    # return reversed list
                    return solution[::-1]
                else:
                    frontier.add(child)

    # TODO
    raise NotImplementedError


**DFS Main Function**

In [ ]:
def main1():
    
    directory = sys.argv[1] if len(sys.argv) == 2 else "large"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path_dfs(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


In [ ]:
if __name__ == "__main__":
    main1()

Loading data...
Data loaded.
Name: Tom Hanks
Name: Tom Cruise
Searching: 6
6 degrees of separation.
1: Tom Hanks and Wendy Makkena starred in A Beautiful Day in the Neighborhood
2: Wendy Makkena and Bradley Whitford starred in The People
3: Bradley Whitford and Katie Holmes starred in A Happening of Monumental Proportions
4: Katie Holmes and Ryan Reynolds starred in Woman in Gold
5: Ryan Reynolds and Kevin Costner starred in Criminal
6: Kevin Costner and Tom Cruise starred in Close Up


From the above two search algorithms, 
- Breadth First Search is better compared to Depth First Search 
- BFS is yielding a degree of separation lesser than that of DFS.